# Extracellular Recording

Extracellular potentials created by given neural networks can be determined at arbitary positions in 3D space. With large neural networks, the potential at a given point is a reasonable approximation to spikes and local field potentials with realistic physiological noise models.

## Simulation 1: Grid of extracellular potentials from a single neuron

In [ ]:
using JNeuron, PyPlot, PyCall;
@pyimport matplotlib.collections as C

#Load 3D Neurolucida file
myimport=input("./data/cell2.asc");

#Generate instance of neuron type with appropriate sections from 3D data
blank_neuron=instantiate(myimport);

#Create segments based on lambda rule
set_nsegs!(blank_neuron);

#add HH and Passive to Soma and Axon, Passive to basal and apical Dendrites
myneuron=add(blank_neuron,(HH(),Passive()));

#Create network with neurons and simulation stop time of 15.0 ms
mynetwork=Network(myneuron,6.0);

#Add electrodes at grid locations
for x in 0.0:250.0:1000.0
    for y in -1000.0:250.0:1500.0
        
        #Create electrode and add it to the network
        electrode=Extracellular([x,y,0.0]);
        add!(mynetwork,electrode);
        
    end
end

#Create intracellular stimulation at soma and add it to the network
mystim=Stim(5.0,1,1,924,1.0,2.0)
add!(mynetwork,mystim)

run!(mynetwork, true);

In [ ]:
xy1=JNeuron.plot_arrays(myneuron)

c1=C.LineCollection(xy1,color="gray",linewidth=2.0, alpha=0.7)

w=Array(Array{Float64,2},length(mynetwork.extra))
rgb=Array(Array{Float64,1},length(mynetwork.extra))

for i=1:length(mynetwork.extra)
    ex=mynetwork.extra[i]
    xs=(ex.xyz[1]-round(Int,length(ex.v)/2)):1.0:(ex.xyz[1]+round(Int,length(ex.v)/2))
    ys=ex.xyz[2]
    if maximum(abs(ex.v))>.01
        w[i]=hcat(xs,ex.v.*3000+ys)
        rgb[i]=[1; 0; 0]
    elseif maximum(abs(ex.v))>.005
        w[i]=hcat(xs,ex.v.*6000+ys)
        rgb[i]=[0; 0; 1]
    else
        w[i]=hcat(xs,ex.v.*9000+ys)
        rgb[i]=[1; 0; 1]
    end
end

c2=C.LineCollection(w,color=rgb,alpha=.7)

fig,ax=subplots(1,1)
ax[:add_collection](c1)
ax[:add_collection](c2)
ax[:set_xlim]([-250.0, 1750.0])
ax[:set_ylim]([-1850.0, 1650.0])

ax[:spines]["top"][:set_visible](false)
ax[:spines]["right"][:set_visible](false)
ax[:spines]["left"][:set_visible](false)
ax[:spines]["bottom"][:set_visible](false)

ax[:set_xticks]([])
ax[:set_yticks]([])

## Simulation 2: Grid of extracellular potentials from multiple neurons

In [ ]:
using JNeuron, PyPlot, PyCall;
@pyimport matplotlib.collections as C

#Load 3D Neurolucida file
myimport=input("./data/cell2.asc");

#Generate instance of neuron type with appropriate sections from 3D data
blank_neuron=instantiate(myimport);

#Create segments based on lambda rule
set_nsegs!(blank_neuron);

#add HH and Passive to Soma and Axon, Passive to basal and apical Dendrites
myneuron1=add(blank_neuron,(HH(),Passive()),(HH(),Passive()),Passive(),Passive());

myneuron2=deepcopy(myneuron1)
JNeuron.translate3d!(myneuron2,400.0,-1000.0,0.0)
JNeuron.randomize_shape!(myneuron2)

myneuron3=deepcopy(myneuron1)
JNeuron.translate3d!(myneuron3,-400.0,-1000.0,0.0)
JNeuron.randomize_shape!(myneuron3)

#Create network with neurons and simulation stop time of 15.0 ms
mynetwork=Network([myneuron1,myneuron2,myneuron3],6.0);

#Add electrodes at grid locations
for x in 0.0:250.0:1500.0
    for y in -1500.0:250.0:1500.0
        
        #Create electrode and add it to the network
        electrode=Extracellular([x,y,0.0]);
        add!(mynetwork,electrode);
        
    end
end

#Create intracellular stimulation at soma and add it to the network
mystim=Stim(5.0,1,1,924,1.0,2.0)
add!(mynetwork,mystim)

mystim=Stim(5.0,1,2,924,2.0,3.0)
add!(mynetwork,mystim)

mystim=Stim(5.0,1,2,924,3.0,4.0)
add!(mynetwork,mystim)

run!(mynetwork);

In [ ]:
#Plot 3D structure of cell
xy1=JNeuron.plot_arrays(myneuron1)
xy2=JNeuron.plot_arrays(myneuron2)
xy3=JNeuron.plot_arrays(myneuron3)

c1=C.LineCollection(xy1,color="gray",linewidth=2.0,alpha=.7)
c2=C.LineCollection(xy2,color="gray",linewidth=2.0,alpha=.7)
c3=C.LineCollection(xy3,color="gray",linewidth=2.0,alpha=.7)

w=Array(Array{Float64,2},length(mynetwork.extra))
alpha=zeros(Float64,length(mynetwork.extra))

for i=1:length(mynetwork.extra)
    ex=mynetwork.extra[i]
    xs=(ex.xyz[1]-round(Int,length(ex.v)/2)):1.0:(ex.xyz[1]+round(Int,length(ex.v)/2))
    ys=ex.xyz[2]
    w[i]=hcat(xs,ex.v.*1900+ys)
    alpha[i]=maximum(abs(ex.v))*3
end

c4=C.LineCollection(w,color="blue",alpha=.7)

fig,ax=subplots(1,1)
ax[:add_collection](c1)
ax[:add_collection](c2)
ax[:add_collection](c3)
ax[:add_collection](c4)
ax[:set_xlim]([-250.0, 1750.0])
ax[:set_ylim]([-1850.0, 1650.0])

ax[:spines]["top"][:set_visible](false)
ax[:spines]["right"][:set_visible](false)
ax[:spines]["left"][:set_visible](false)
ax[:spines]["bottom"][:set_visible](false)

ax[:set_xticks]([])
ax[:set_yticks]([])

In [ ]:
plot(mynetwork.extra[30].v)